In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import nltk
import re
import os
import codecs
from sklearn import feature_extraction


In [ ]:
!pip install mpld3

In [ ]:
import mpld3

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/UTS PIMO/question_philoit.csv',usecols=['content','created_at'],parse_dates=['created_at'], index_col="created_at")
df

,content
created_at,
2019-04-22 21:14:14,Apa saja yang perlu dipersiapakan apabila indo...
2019-04-23 19:36:32,Apa saja contoh penggunaan artificial intellig...
2019-04-24 18:59:06,Mengapa Go-Jek diinvestasi oleh asing bukan pe...
2019-04-30 09:48:54,Apa kelebihan dan kekurangan masing-masing 3 f...
2019-05-09 10:05:23,Apakah blockchain memungkinkan untuk diimpleme...
...,...
2020-09-21 08:09:31,apa yang menyebabkan terjadinya banjir
2020-09-21 08:13:23,Apa manfaat mengonsumsi ikan mas
2020-09-21 08:15:34,apakah ada kalian yang menggunakan aplikasi in...


In [ ]:
df.shape

(91163, 1)

Seleksi tanggal pertanyaan dalam bulan
Feb-Apr 2020

In [ ]:
startyy, startMM, startdd = (2020,2,1) # 1 Feb 2020
endyy, endMM, enddd = (2020, 4,30) # 30 apr 2020

month_day = pd.concat([
                df.index.to_series().dt.year,
                df.index.to_series().dt.month, 
                df.index.to_series().dt.day ], axis=1).apply(tuple, axis=1)

df[(month_day >= (startyy, startMM, startdd)) & (month_day <= (endyy, endMM, enddd))]

,content
created_at,
2020-02-01 00:37:00,Apa yang seharusnya kita kerjakan saat liburan?
2020-02-01 00:41:34,Apa makanan khas daerah kalian ?
2020-02-01 00:41:34,Apa makanan khas daerah kalian ?
2020-02-01 00:41:34,Apa makanan khas daerah kalian ?
2020-02-01 01:25:41,Menurut kamu yang akan juara club mana ditahun...
...,...
2020-04-29 08:26:15,Apa saja kemudahan kalian bekerja dan belajar ...
2020-04-29 08:26:39,Bagaimana cara kalian agar tidak bosan selama ...
2020-04-29 08:41:07,Apa olahraga yang mudah dilakukan selama di ru...


In [ ]:
df[(month_day >= (startyy, startMM, startdd)) & (month_day <= (endyy, endMM, enddd))].shape

(3140, 1)

tokenisasi masih bahasa inggris

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [ ]:
def tokenize_and_stem(row):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(row) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(row):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(row) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in df['content']:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

TF IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(df['content']) 

print(tfidf_matrix.shape)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


CPU times: user 31 s, sys: 229 ms, total: 31.3 s
Wall time: 31.3 s
(91163, 2)


In [ ]:
terms = tfidf_vectorizer.get_feature_names()

In [ ]:
#terjadi crash
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

Kmeans

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 5
km = KMeans(n_clusters=num_clusters)
%time km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

CPU times: user 3.71 s, sys: 23.1 ms, total: 3.73 s
Wall time: 3.82 s
